In [ ]:
import nltk
import sqlite3
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd





In [ ]:
print("Loading Data From DB")
# db must be in same folder as notebook
targetDatabase = 'engA.db'
con = sqlite3.connect(targetDatabase)
db = pd.read_sql_query("""SELECT sid, cid, clemma, tag, tags FROM concept""",con)

print("Data Loaded!")



In [ ]:
# Check shape of db
db.head(5)


In [ ]:
# for each word, how many instances
#  for each word instance, list tags
#   for each tags, how many times the tag appear in the word

def tagsCount(row):
    tagsString = row['tags']
    if tagsString == None:
        return 0
    else:
        return len(row['tags'].split("; "))

db['tagsCount'] =  db.apply(tagsCount, axis=1)
# db.head(20)
print("Done!")

In [ ]:
numWords = db.groupby('clemma').size()
wordsDB = db.drop(['sid', 'cid','tag'], axis = 1)
numWords = numWords.to_frame()
numWords.reset_index(level=0, inplace=True)
numWords.rename(columns={ numWords.columns[0]: "clemma" , numWords.columns[1]: 'wordCount'}, inplace = True)
numWords.head(5)
print("Done!")

In [ ]:
allTags = db.drop(['clemma','sid', 'cid','tags','tagsCount'], axis = 1)
allTags = allTags.drop_duplicates()

allWords = db.drop(['tag','sid', 'cid','tags','tagsCount'], axis = 1)
allWords = allWords.drop_duplicates()

allWordCounts = db.groupby(['clemma']).size()
print("Done!")

In [ ]:

wordsDB = pd.merge(wordsDB,numWords, on="clemma")
wordsDB = wordsDB.drop_duplicates()
def stringToList(string):
    if string == None:
        return []
    else:
        return string.split('; ')

wordsDB['tags'] = wordsDB.apply(lambda x : stringToList(x['tags']), axis =1  )
a= wordsDB.groupby(['clemma']).agg({'tags': "sum"})
a.reset_index(level=0, inplace=True)
a['tags'] = a.apply(lambda r: set(r['tags']), axis = 1)
a['tagsCount'] = a.apply(lambda r: len(r['tags']), axis = 1)
a = a.sort_values(['tagsCount'], ascending = False )

In [ ]:
tagCount=0

def getTagCount(row):
    global tagCount
    tagCount+=1      
    word = row['clemma']
    print(word, tagCount*100/len(allWords), "percent Complete")
    tags = dict.fromkeys(row['tags'], 0)
    strippedTags = {}
    s = 0
    sumTags=0
    tags['w'] = 0
    tags['x'] = 0
    tags['pers'] = 0
    for tag in tags.keys():
        # TODO: CHANGED HERE
        strippedTag = tag.strip()
        if tag != strippedTag:
            print(f"'{tag}', '{strippedTag}'")
        count = len(db[ (db['clemma'] == word) & ( db['tag'] == strippedTag) ].index)
        strippedTags[strippedTag] = count
        sumTags+=count
        n = count/allWordCounts[word]
        if n == 0:
            continue
        nlogn = n * math.log10(n)
        s += abs(nlogn)

    if (s>1):
        print("ANOMALY!", word )
        print("word Count:",allWordCounts[word], "tag: ",sumTags )
    return (s,strippedTags)

b  = a.apply(getTagCount, axis =1)
        

In [ ]:
# MUST EXECUTE
a["results"] = b
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a

In [ ]:
# OPTIONAL
# To check a specific clemma
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_colwidth', -1)
a[ (a['clemma'] == "break" )]

In [ ]:
# To save results
import os
print(os.getcwd())
p = os.path.join(os.getcwd(), "PicklepeePumperrump.pckl")
print(p)
a.to_pickle(p)
# Remember to change name of pckl file

In [ ]:
# To read results
print("Unpickling!")

rdb = pd.read_pickle('2019hg8011_engA_processed.pckl')
print("Unpickled!")
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', 0)
rdb

In [ ]:
# To sort results
rdb['entropyVal'] =  rdb.apply(lambda x: x['results'][0], axis =1)
rdb = rdb.sort_values(['entropyVal'],axis = 0, ascending= True )
rdb.head(1903)

In [ ]:
rdb = rdb.sort_values(['entropyVal'],axis = 0, ascending= False )
rdb.head(10)

In [ ]:
# Excel conversion
rdb.to_excel("output.xlsx")